# Import packages 

In [1]:
# standard packages
from utils.pickling import pickleIt
from utils.plotting import plotIt
from utils.metrics import SharpeRatio, MDD
from universal import algos
import seaborn as sns
from DeepRL.utils.misc import run_episodes
import shutil
import pickle
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import torch
from networks.base_networks import DeterministicActorNet, DeterministicCriticNet
from DeepRL.component import GaussianPolicy, HighDimActionReplay, OrnsteinUhlenbeckProcess
from DeepRL.agent import ProximalPolicyOptimization, DisjointActorCriticNet
import gym
from DeepRL.utils import Logger
from agents.DDPGAgent import DDPGAgent
from environment.config import Config
from wrappers import TransposeHistory, ConcatStates, SoftmaxActions, DeepRLWrapper
from environment.stock_environment import PortfolioEnv
import argparse
from utils.dataPreprocessing import preProcessData
import datetime
import logging
import matplotlib.pyplot as plt
import os
import time
import pandas as pd
import numpy as np
import matplotlib as mpl

/Users/stewart/miniconda3/envs/rl_pm/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


ImportError: No module named 'network'

# Pre-define some utility functions 

In [3]:
def save_ddpg(agent):
    agent_type = agent.__class__.__name__
    save_file = 'data/%s-%s-model-%s.bin' % (agent_type, config.tag, agent.task.name)
    agent.save(save_file)
    print(save_file)
    

def load_ddpg(agent):
    agent_type = agent.__class__.__name__
    save_file = 'data/%s-%s-model-%s.bin' % (agent_type, config.tag, agent.task.name)
    new_states = pickle.load(open(save_file, 'rb'))
    states = agent.worker_network.load_state_dict(new_states)


def load_stats_ddpg(agent):
    agent_type = agent.__class__.__name__
    online_stats_file = 'data/%s-%s-online-stats-%s.bin' % (
                    agent_type, config.tag, agent.task.name)
    try:
        steps, rewards = pickle.load(open(online_stats_file, 'rb'))
    except FileNotFoundError:
        steps =[]
        rewards=[]
    df_online = pd.DataFrame(np.array([steps, rewards]).T, columns=['steps','rewards'])
    if len(df_online):
        df_online['step'] = df_online['steps'].cumsum()
        df_online.index.name = 'episodes'
    
    stats_file = 'data/%s-%s-all-stats-%s.bin' % (agent_type, config.tag, agent.task.name)
    try:
        stats = pickle.load(open(stats_file, 'rb'))
    except FileNotFoundError:
        stats = {}
    df = pd.DataFrame(stats["test_rewards"], columns=['rewards'])
    if len(df):
#         df["steps"]=range(len(df))*50

        df.index.name = 'episodes'
    return df_online, df

def universalPortfolioStrat(env, algo, seed=0):
    env.seed(0)
    np.random.seed(0)
    # start the environment from the start using reset()
    state = env.reset()
    # unwrapped removes all wrappers the environment instance has, 
    # then step through the environment with the for loop
    for _ in range(env.unwrapped.sim.steps):
        history = pd.DataFrame(state[0,:,:], columns=env.unwrapped.src.asset_names)
        # modern portfolio theory approach to universal portfolios needs cash as 1st column
        history["CASH"] = 1
        history = history[['CASH'] + env.unwrapped.src.asset_names]
        # some strategies need a history - history (BSF, OLMAR), 
        # others just need the previous time step - x (ONS, EG, RMR)
        x = history.iloc[-1]
        # portfolio weights w0 from the previous time step
        last_weights = env.unwrapped.sim.w0
        # fill algo object with stock returns history
        algo.init_step(history)
        # some strategies don't require history
        try:
            action = algo.step(x, last_weights, history)
        except TypeError:
            action = algo.step(x, last_weights)
        action = getattr(action, 'value', action)
        # format for universal portfolio theory strategy
        if isinstance(action, np.matrixlib.defmatrix.matrix):
            action = np.array(action.tolist()).T[0]
        # take the action on the environment, observing new state and reward
        # done=at last time step (True), o.w. (False), info=debugging dictionary
        state, reward, done, info = env.step(action)
        # if at last time step, break out of for loop
        if done:
            break
    # make dataframe of the returns information
    df = pd.DataFrame(env.unwrapped.infos)
    df.index = pd.to_datetime(df['date']*1e9)
    
    return df['portfolio_value'], df

# Access some financial data 

**Please note, I used thomson reuters datastream / refinitiv data for my dissertation, but I cannot share such data in any format outside of academic use**